In [2]:
from google.cloud import bigquery
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import datetime
from google.cloud import bigquery
import pytz
from google.oauth2 import service_account
import numpy as np
import requests
from datetime import datetime as dt
from datetime import date

In [3]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'.\creds\opendatadbt-f2e674527eb6.json'
client = bigquery.Client()

credentials = ServiceAccountCredentials.from_json_keyfile_name(r'.\creds\opendatadbt-f2e674527eb6.json') # Your json file here


#Initial Load

austin311 = requests.get("https://data.austintexas.gov/resource/xwdj-i9he.json?$limit=50000&$where=sr_created_date>'2021-03-24'" ) 
austin311json = austin311.json()
cols = ['sr_number','sr_created_date','sr_closed_date','sr_type_desc','sr_location', 'sr_location_zip_code','sr_location_city','sr_status_desc','sr_location_lat','sr_location_long']
austin_df = pd.DataFrame(austin311json, columns=cols)
austin_df

In [4]:
#Incremental load
austin_table_id = "opendatadbt.311.austin311"
current_datetime = dt.today().strftime("%Y-%m-%dT%H:%M:%S.000")
query_string = """
SELECT 
max(sr_created_date), max(sr_closed_date)
FROM `opendatadbt.311.austin311`
limit 10
"""

austinmaxopendatedataframe = (
    client.query(query_string)
    .result()
    .to_dataframe()
)
austinmaxopendatedate = str(austinmaxopendatedataframe['f0_'][0])

austinmaxclosedatedataframe = (
    client.query(query_string)
    .result()
    .to_dataframe()
)
austinmaxclosedate = str(austinmaxclosedatedataframe['f1_'][0])


In [6]:

austin311 = requests.get("https://data.austintexas.gov/resource/xwdj-i9he.json?$limit=50000&$where=sr_created_date>'"+austinmaxopendatedate+"' OR sr_closed_date between '" + austinmaxclosedate+ "' AND '" + current_datetime + "'") 
austin311json = austin311.json()
cols = ['sr_number','sr_created_date','sr_closed_date','sr_type_desc','sr_location', 'sr_location_zip_code','sr_location_city','sr_status_desc','sr_location_lat','sr_location_long']
austin_df = pd.DataFrame(austin311json, columns=cols)
austin_df

,sr_number,sr_created_date,sr_closed_date,sr_type_desc,sr_location,sr_location_zip_code,sr_location_city,sr_status_desc,sr_location_lat,sr_location_long
0,21-00145042,2021-03-26T16:13:10.000,2021-03-26T21:26:14.000,Debris in Street,"2404 E 14TH ST, AUSTIN, TX 78702",78702,AUSTIN,Resolved,30.27661733,-97.71512247
1,21-00145414,2021-03-26T19:00:16.000,2021-03-28T02:11:52.000,Loose Dog,"14716 HYSON XING, AUSTIN, TX 78660",78660,AUSTIN,Open,30.42333634,-97.63802765
2,21-00143969,2021-03-26T10:37:01.000,2021-03-27T02:11:52.000,ARR Missed Yard Trimmings/Compost,"4401 NORWOOD LN, AUSTIN, TX 78744",78744,AUSTIN,New,30.18815671,-97.68762169
3,21-00143987,2021-03-26T10:40:53.000,2021-03-27T02:11:52.000,Street Light Issue- Address,"10524 THOROUGHBRED DR, AUSTIN, TX 78748",78748,AUSTIN,Open,30.15350009,-97.80551811
4,21-00145445,2021-03-26T19:23:22.000,2021-03-27T00:09:30.000,Debris in Street,"MC KALLA PL & RUTLAND DR, AUSTIN, TX",78758,AUSTIN,Resolved,30.38240456,-97.72096214
...,...,...,...,...,...,...,...,...,...,...
1475,21-00144277,2021-03-26T11:57:19.000,2021-03-27T02:11:52.000,ARR Missed Yard Trimmings/Compost,"11602 RYDALWATER LN, AUSTIN, TX 78754",78754,AUSTIN,New,30.35384924,-97.61392235
1476,21-00144920,2021-03-26T15:28:22.000,2021-03-27T02:11:52.000,Speed Management,"305 PROVINES DR, AUSTIN, TX 78753",78753,AUSTIN,New,30.37528832,-97.68612782
1477,21-00145610,2021-03-26T22:15:54.000,2021-03-27T09:04:37.000,Traffic Signal - Maintenance,"W WILLIAM CANNON DR & W US 290 HWY, AUSTIN, TX",78749,AUSTIN,Closed,30.23405521,-97.86483461
1478,21-00145612,2021-03-26T22:20:46.000,2021-03-27T18:12:10.000,Loud Commercial Music,"8912 N LAMAR BLVD, AUSTIN, TX 78758",78758,AUSTIN,Closed,30.35850635,-97.69996779


In [7]:
austin_table_id = "opendatadbt.311.austin311"
dataset_ref = client.dataset('311')
table_ref = dataset_ref.table('austin311')
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",
)

client.load_table_from_dataframe(
    austin_df, table_ref, job_config=job_config
).result()  # Make an API request.
